In [1]:
# !wget "https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/Json%20Format%20of%20Persian%20Wikipedia%20Pages/fawiki-20181001-pages-articles-multistream%201%20-%20100000.zip"
# !wget "https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/Json%20Format%20of%20Persian%20Wikipedia%20Pages/fawiki-20181001-pages-articles-multistream%20100001%20-%20290169.zip"
# !wget "https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/Json%20Format%20of%20Persian%20Wikipedia%20Pages/fawiki-20181001-pages-articles-multistream%20290170%20-%20580338.zip"
# !wget "https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/Json%20Format%20of%20Persian%20Wikipedia%20Pages/fawiki-20181001-pages-articles-multistream%20580339%20-%20870507.zip"
# !wget "https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/Json%20Format%20of%20Persian%20Wikipedia%20Pages/fawiki-20181001-pages-articles-multistream%20870508%20-%201160676.zip"

In [2]:
# !unzip "/kaggle/working/fawiki-20181001-pages-articles-multistream 1 - 100000.zip"
# !rm "/kaggle/working/fawiki-20181001-pages-articles-multistream 1 - 100000.zip" 

# !unzip "/kaggle/working/fawiki-20181001-pages-articles-multistream 870508 - 1160676.zip"
# !rm "/kaggle/working/fawiki-20181001-pages-articles-multistream 870508 - 1160676.zip" 

# !unzip "/kaggle/working/fawiki-20181001-pages-articles-multistream 100001 - 290169.zip"
# !rm "/kaggle/working/fawiki-20181001-pages-articles-multistream 100001 - 290169.zip" 

# !unzip "/kaggle/working/fawiki-20181001-pages-articles-multistream 290170 - 580338.zip"
# !rm "/kaggle/working/fawiki-20181001-pages-articles-multistream 290170 - 580338.zip" 

# !unzip "/kaggle/working/fawiki-20181001-pages-articles-multistream 580339 - 870507.zip"
# !rm "/kaggle/working/fawiki-20181001-pages-articles-multistream 580339 - 870507.zip" 

In [3]:
# !wget "https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11234/1-3195/lscp-0.5-fa.7z"

In [4]:
# !7z x "/kaggle/working/lscp-0.5-fa.7z"
# !rm "/kaggle/working/lscp-0.5-fa.7z" 

In [5]:
# !wget "https://raw.githubusercontent.com/amirmosio/ColloquialWordProject/main/language_model/data/stop_words.json"

In [6]:
# !wget "https://github.com/Text-Mining/Persian-Wikipedia-Corpus/raw/master/models/word2vec-cbow/word2vec.model-cbow-size%3D200-window%3D5.part1.rar"
# !wget "https://github.com/Text-Mining/Persian-Wikipedia-Corpus/blob/master/models/word2vec-cbow/word2vec.model-cbow-size%3D200-window%3D5.part2.rar"
# !wget -o "/kaggle/working/word2vec.model" "https://drive.google.com/uc?id=1M6aG47Kxm_kw1ADtDm1cXaOLZ_qM0qys&export=download"

In [7]:
import os
# os.remove("./word2vec.model1668043951.0301144.model.1")
# os.remove("./fawiki-20181001-pages-articles-multistream 870508 - 1160676.json")

for dirname, _, filenames in os.walk('../input/train-word2vec-for-one-epoch'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/train-word2vec-for-one-epoch/stop_words.json
../input/train-word2vec-for-one-epoch/lscp-0.5-fa.txt
../input/train-word2vec-for-one-epoch/formal_tokens.json
../input/train-word2vec-for-one-epoch/word2vec.model1668043951.0301144.model.syn1neg.npy
../input/train-word2vec-for-one-epoch/word2vec.model1668043938.615096.model.wv.vectors.npy
../input/train-word2vec-for-one-epoch/fawiki-20181001-pages-articles-multistream 870508 - 1160676.json
../input/train-word2vec-for-one-epoch/fawiki-20181001-pages-articles-multistream 100001 - 290169.json
../input/train-word2vec-for-one-epoch/fawiki-20181001-pages-articles-multistream 290170 - 580338.json
../input/train-word2vec-for-one-epoch/word2vec.model1668043938.615096.model.syn1neg.npy
../input/train-word2vec-for-one-epoch/word2vec.model1668043951.0301144.model
../input/train-word2vec-for-one-epoch/fawiki-20181001-pages-articles-multistream 1 - 100000.json
../input/train-word2vec-for-one-epoch/word2vec.model1668043951.0301144.model.wv.vector

In [8]:
import torch


class Configs:
    DEBUG = False
    question_answer_score = 1
    question_wrong_answer_score = -5
    introduce_score = 20
    music_cost = 3
    bot_user_name = "fa_cwc_bot"
    min_score_for_lottery = 100
    test_context_number = 100
    train_epochs = 1
    train_max_queue_length = 1000
    device = torch.device("cuda" if torch.cuda.is_available() and not DEBUG else "cpu")
    print(f"device: {device}")



device: cpu


In [9]:
!pip install hazm
import json
import re
import string
import time
from hazm import word_tokenize, Lemmatizer
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from tqdm import tqdm_notebook as tqdm
from itertools import chain, islice
import numpy as np
from transformers import BertTokenizer, BertForMaskedLM, pipeline, AdamW

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=22ee967338a2c2deed3f9ec2cd17bbdd49f36a219553f281d245c0deada5968f
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=192764 sha256=340c428948186fb50d3b0d8c6203e3f819b2b123de5bf4e12ef93e5ce6af9c15
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      

In [10]:
class PreprocessUtilities:
    lemmatizer = Lemmatizer()
    punctuation = string.punctuation + '،؟'

    def __init__(self, stop_words):
        self.stop_words = stop_words if stop_words else []
        self.end_of_sentence_sings = [".", "?", "؟", "!", ";", "؛", ":", "\n"]

        self.pre_processing_functions = [self.remove_emoji_from_text,
                                         self.remove_english_char_from_text,
                                         self.remove_signs,
                                         self.remove_digits_and_replace_digits_sign,
                                         self.remove_url_from_text,
                                         self.remove_extra_spaces,
                                         self.remove_half_space,
                                         self.remove_inflections]

    def split_sentences(self, text):
        return re.split(f"|".join(["\\" + i for i in self.end_of_sentence_sings]), text)

    @staticmethod
    def remove_emoji_from_text(text):
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   "]+", flags=re.UNICODE)
        return emoji_pattern.sub(r' ', text)

    @staticmethod
    def remove_url_from_text(text):
        return re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)

    @staticmethod
    def remove_digits_and_replace_digits_sign(text):
        return re.sub(r'[0-9|۰-۹]+', '#Digits#', text, flags=re.MULTILINE)

    @staticmethod
    def remove_english_char_from_text(text):
        return re.sub(r'[a-zA-Z]', ' ', text, flags=re.MULTILINE)

    @staticmethod
    def remove_signs(text):
        for char in PreprocessUtilities.punctuation:
            text = text.replace(char, ' ')
        return text

    @staticmethod
    def remove_extra_spaces(text):
        return re.sub(r'\s+', ' ', text, flags=re.MULTILINE)

    def remove_stop_words(self, tokens):
        res = []
        for token in tokens:
            if token not in self.stop_words:
                res.append(token)
        return res

    @staticmethod
    def remove_half_space(text):
        return re.sub(r'\u200c', ' ', text, flags=re.MULTILINE)

    @staticmethod
    def remove_inflections(text):
        tokens = word_tokenize(text)
        lemma_tokens = [PreprocessUtilities.lemmatizer.lemmatize(w) for w in tokens]
        return " ".join(lemma_tokens)

    @staticmethod
    def tokenize_text(text):
        return text.split()

    def process_context(self, context, remove_stop_words=True):
        for proc in self.pre_processing_functions:
            context = proc(context)
        if remove_stop_words:
            return self.remove_stop_words(self.tokenize_text(context))
        return self.tokenize_text(context)

In [11]:
class FormalAndColloquialDataPreProcessing:
    formal_directory_path = "language_model/data/formal_dataset/"
    formal_file_paths = ['fawiki-20181001-pages-articles-multistream 1 - 100000.json',
                         'fawiki-20181001-pages-articles-multistream 100001 - 290169.json',
                         'fawiki-20181001-pages-articles-multistream 290170 - 580338.json',
                         'fawiki-20181001-pages-articles-multistream 580339 - 870507.json',
                         'fawiki-20181001-pages-articles-multistream 870508 - 1160676.json']
    colloquial_file_path = 'language_model/data/colloquial_dataset/lscp-0.5-fa.txt'
    stop_words_file_path = 'language_model/data/stop_words.json'

    def __init__(self):
        with open(self.stop_words_file_path, 'rb') as file:
            stop_words = json.loads(file.read())['stopWords']
        self.preprocess_utils = PreprocessUtilities(stop_words)

    def bring_processed_formal_tokens(self, file_index: int):
        for context in self._load_raw_formal_data(file_index):
            yield self.preprocess_utils.process_context(context)

    def bring_processed_colloquial_tokens(self):
        for context in self._load_raw_colloquial_data():
            yield self.preprocess_utils.process_context(context)

    def _load_raw_formal_data(self, file_index):
        with open(self.formal_directory_path + self.formal_file_paths[file_index], 'rb') as file:
            context_count = 0
            for article in file:
                article = article.decode('utf-8')
                if article != "":
                    for context in json.loads(article)['Text'].split("\n"):
                        context_count += 1
                        yield context
                    if Configs.DEBUG and context_count >= Configs.test_context_number:
                        break

    def _load_raw_colloquial_data(self):
        with open(self.colloquial_file_path, 'rb') as file:
            context_count = 0
            for context in file:
                context = context.decode('utf-8').strip()
                if context.startswith("RT : "):
                    context = context[5:]
                context_count += 1
                yield context
                if Configs.DEBUG and context_count >= Configs.test_context_number:
                    break

In [12]:
FormalAndColloquialDataPreProcessing.formal_directory_path = "../input/train-word2vec-for-one-epoch/"
FormalAndColloquialDataPreProcessing.colloquial_file_path = '../input/train-word2vec-for-one-epoch/lscp-0.5-fa.txt'
FormalAndColloquialDataPreProcessing.stop_words_file_path = '../input/train-word2vec-for-one-epoch/stop_words.json'

In [13]:
# provider = FormalAndColloquialDataPreProcessing()
# data = provider._load_raw_colloquial_data()

# for context in tqdm(data):
#     print(context)
#     for proc in provider.pre_processing_functions:
#         context = proc(context)
#     print(provider.preprocess_utils.remove_stop_words(provider.tokenize_text(context)))

In [14]:
class SentencesIterator:
    def __init__(self, generator_function):
        self.generator_function = generator_function
        self.generator = self.generator_function()

    def __iter__(self):
        # reset the generator
        self.generator = self.generator_function()
        return self

    def __next__(self):
        result = next(self.generator)
        if result is None:
            raise StopIteration
        else:
            return result


class Word2VecLanguageModelService:
    model_path_save = "language_model/word2vec.model"
    model_path_load = "language_model/word2vec.model19308.model"
    formal_tokens_path_save = "language_model/formal_tokens.json"
    formal_tokens_path_load = "language_model/kaggle_outputs/formal_tokens_merged_with_bert.json"

    class CallBack(CallbackAny2Vec):
        def __init__(self):
            self.epoch = 0

        def on_train_begin(self, model):
            print('Training began ..., it may take a while!')
            pass

        def on_epoch_end(self, model):
            self.epoch += 1
            print(str(self.epoch) + "\\" + str(model.epochs), end=" - " if self.epoch != model.epochs else "\n")
            # model.save(Word2VecLanguageModelService.model_path_save)

    def __init__(self, load_model=True):
        self.provider = FormalAndColloquialDataPreProcessing()
        self.model = self.load_model() if load_model else None
        self.formal_tokens = self.load_formal_tokens_json() if load_model else set()

    def load_model(self):
        return Word2Vec.load(self.model_path_load)

    def save_model(self):
        print(f"Saving model...")
        day = int(time.time() / (24 * 60 * 60))
        self.model.save(self.model_path_save + str(day) + ".model")
        print("Saved.")

    def train_new_text(self, corpus_iterable, epoch=None):
        print(f"Training started on {len(corpus_iterable)} ...")
        self.model.min_count = 1
        self.model.train(corpus_iterable, total_examples=len(corpus_iterable), epochs=epoch)
        self.model.build_vocab(corpus_iterable, update=True)
        print(f"Training finished")

    @staticmethod
    def shuffle(generator, buffer_size=4000):
        count_txt = 0
        while True:
            buffer = list(islice(generator, buffer_size))
            if len(buffer) == 0:
                break
            np.random.shuffle(buffer)
            for item in buffer:
                count_txt += 1
                if count_txt % 10000 == 0:
                    print(10000, end="-")
                yield item

    def train_model(self):
        def get_generator():
            all_tokens_generator = chain(self.provider.bring_processed_colloquial_tokens(),
                                         self.provider.bring_processed_formal_tokens(0),
                                         self.provider.bring_processed_formal_tokens(1),
                                         self.provider.bring_processed_formal_tokens(2),
                                         self.provider.bring_processed_formal_tokens(3),
                                         self.provider.bring_processed_formal_tokens(4))
            return self.shuffle(all_tokens_generator)

        self.model = Word2Vec(sentences=SentencesIterator(get_generator), vector_size=200, window=5, workers=4,
                              epochs=Configs.train_epochs,
                              callbacks=(self.CallBack(),), min_count=1)
        self.save_model()

    """
    Formal Tokens List
    """

    def process_all_formal_tokens_and_save(self):
        for i in tqdm(range(5)):
            for context in tqdm(self.provider.bring_processed_formal_tokens(i)):
                if Configs.DEBUG:
                    for word in ['جوریم', 'دیوونم', 'دیوونس', 'خونس']:
                        if word in context:
                            print("found one!")
                            breakpoint()
                self.formal_tokens.update(context)

        with open(self.formal_tokens_path_save, 'w') as file:
            file.write(json.dumps(list(self.formal_tokens)))

    @classmethod
    def load_formal_tokens_json(cls):
        with open(cls.formal_tokens_path_load, 'rb') as file:
            formal_tokens = json.loads(file.read())
        return set(formal_tokens)

    def get_similar_words_from_formal_or_both(self, positive=None, negative=None, topn=9, just_return_formals=True):
        try:
            res = {}
            coeff_constant = 4  # maybe a good estimates for colloquial_tokens/formal_tokens
            while len(res) != topn and coeff_constant < 2048:
                for token, distance in self.model.wv.most_similar(positive=positive, negative=negative,
                                                                  topn=topn * coeff_constant):
                    if token not in res and (not just_return_formals or token in self.formal_tokens):
                        res[token] = round(1 / distance, 3)
                        if len(res) == topn:
                            break
                coeff_constant *= 2
            return res
        except Exception as e:
            print(f"Error in word2vec get_similar_words {e}")
            return None

    """
    Unknown Colloquial Tokens
    """

    def get_back_interesting_token_and_similar_words(self, text):
        text_tokens = self.provider.preprocess_utils.process_context(text)[0]
        for token in text_tokens:
            if token not in self.formal_tokens:
                sim_words = self.get_similar_words_from_formal_or_both(token, topn=9)
                if sim_words:
                    return token, sim_words
        return None, None


In [15]:
# Word2VecLanguageModelService.model_path_load = "../input/train-word2vec-for-one-epoch/word2vec.model"
# Word2VecLanguageModelService.formal_tokens_path_load = "../input/train-word2vec-for-one-epoch/formal_tokens.json"

In [16]:
# wv_service = Word2VecLanguageModelService(load_model=False)
# wv_service.train_model()

In [17]:
# !wget "https://www.kaggleusercontent.com/kf/110334917/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..zCRVqUG8lLD_l0-LFobTCQ.cAaAezHgfqNUWYbIDsfiQ3qsiB30jH1DtLGuUp6EraHDrKZw5TppyTZ0FjyDHx7-YMAbzm_x8P3KHNdHQan47w0d991UgCk6Kb_7Iab1azGhIaEJYqAixUP5_V1i41mdCx2rObh-s68p7lrgCQxmUOnKcA_lU2ejk7VzNZDkwMTrAY5A_3XB9TWrC0re5otJSVYW7-SafyJFQIboAlKqJhHrxwGS8OXMBgGafDDU-89FkKTGpkx8aDQSsJZj2bU1gW72rThnlp5-yqm-48i-vOmObBXFniW3aDG2ZqrEhNtqy31EM0elY5Pnfai4iA2z8l1iiYbPlYBV8Lie_tiXyr22F-dwl4wvy7xnFWcaVIw-kbI9DcA1gNl4tN0WUrltpbFnm7IH8rMTKML4zVOneY7oSWlIAjAuB-_k0ZdihtuJqbR1Fa5unqxBnqV7g4MCzmU3XuxZR-Nr7Gx9_GBxXTSRcyPXCFmcUAYAHlhG8LsVCJDu9L6LgmTEtbPS8Hm68DJesWcLa7CfigtB1CZEZOAn4Jl3tRhrDj-GqYSepTNITFy4HISHD7ZbjrCx95J7nWzw22fLrYuDiEWbGPPxXmI8mgusOyS7Q67u1ly4yHv7d8BxwGr73uQ2-5Rrnu0YD0bk2XywNYMYetQA7kka9w.odRtcbFyzPuu1At2h85wJQ/word2vec.model1667892185.4770463.model"
# !wget "https://www.kaggleusercontent.com/kf/110334917/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..wNvCWEjzdMhQ7cHmc33wCA.fq2TIRSPoDD2i-e6m8Y2D_EMSX6ZebdfenWl74LChZl9k-HX3bU1rL3Wj5L3GQxCwo-9jpFsuPpom2QM5q5or83uWrV3-N11tVlN5NKSsV9IaLwpxzD7tknRB9MYmNgfIEMwXG5DQawNj8Fe8qXrZRZlYslgs95Nvje3NML9pq8YbkiHpp80UmwnLWeGj8cLaEFaxGOGDGhGpfkq1iPu24BRQvNC77KsoV95MQEj6sz01HFCVGHMRuEe7MA3H43sSvTeGCgn63hyEholWZ1HGyHtiwb7gky1lA_zLO2BYouVc2BqK08X9WR9nRq3SuaFRCqYbibevGpwPGYCPVViYEfhhrv4dcTzyWgQGHdl5NHxZiWdYLtrFFownb4Ht-dO_N8f84dE7JKMKKISyEpb1QsqX9SrljxJBFwfXcZ911IBBz1jRm3io0s94ICOUu0fSxLWlHUrL4hjR-jZ-cU2OWlZsYGWfYAG-11gJO2rEr0YPRdFipFCPwnlXiaFc9w25017MoLJ2ir_RLQTKK5LmCmdF_scnpK9yXclYgAAX8tN8PFB1R4vYKiT-VXvpN15KbCPN24GAtkri8BJWkU-LfddR3Lu_xbs28XSoMt8jMiX0k82BeNXVs0s4UkiecCnyP7_Zmd6ggox3pXiOUgRXA.bY5SASfpMMKsst19vYOLiw/word2vec.model1667892185.4770463.model.syn1neg.npy"
# !wget "https://www.kaggleusercontent.com/kf/110334917/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..svEA3Jc0LtUP-It22lyHZA.hwkZd1Vica8FHJhRDTSvm9O62Inpf7q58v7SFohWpjH7ZxOXeZLrYpG-1GophvI7E_V19YA6HtGTLL4wUIzL5xvXeDa981gDougGy5AJnjA8pu2hz8vKTDF1EAXASQ59jrip9aBQjiuqEeOfwjqWzEwafBfCcD07h91uztj7qDzpfMkOIM9K9bJyGufzl76NT8e9a9nvoBDZuUTF3YMQ72fUuse6BeEehYAMVmEw5t4nX9Dr_EiWq2Arr-hGTf9W5iSbvWdJ0xlOU6LdbjEICKGRgSZewfOyjiforyLMviu_vRGzRk6V5gPC0uZpW1dYORTebRowd-RQXSu9BMAUHuT0j_80ZUGsd9eok2DUA7mm75YIesd3tvl2dV9duSqq_U7NdlqS0w6DuktdKaOBIv4c4mdCaudxOi7qpZ6pSIPTszP9Fj0IvTcjo3AG06JLp4SbNTFI4o9E5uaLqcSJ3QWnDFyKO-obLw1daRungn-tJdnLJ_kJQ6HCFEifowbfNcJPGpFZCCJgA4U-9CylymXYgJK4JnaBYPQhMpfG9hYqvQySmCyH2gZdC135jSnIisZ6EaeFAbs351rIGkE0aidmriGXJhnhruNq4yA_YK7hCQHD1MOdLSO-ViPHGQsdeXfGXL4_Ui7Vz8m_YXS_Xg.L6jJzTFkrc-M1jM3lj-7sA/word2vec.model1667892185.4770463.model.wv.vectors.npy"

In [18]:
# Word2VecLanguageModelService.formal_tokens_path = "/kaggle/working/formal_tokens.json"
# word2vec_service = Word2VecLanguageModelService(load_model=False)
# word2vec_service.process_all_formal_tokens_and_save()

In [19]:
# !wget "https://www.kaggleusercontent.com/kf/110519903/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..9jax_tyR6KQla8Pt5BkkNg.9n6Ji8fr2VBN8m2bkBTrES99G5O14MhxabbSltX0HflGapris0YoPVwAXY17xa1paFE-XUkPBOGCznx948r62tC1GvPmuXoZhh37jjy7URsF09bp4O4o7qu5vW6tvuJ7wIR8FR_JLycEembTzFAWa0rlDukD1VEKHhxiaUqEZ8M0O7I0fmbJNWKV1lRX_2oP2JP2TsQ-uqWTvaetGK_BPNThy2GOX6qMMSg5BHeRiOTxLxWEyUBdh9lAg64PzYa2s3NmdNcl8QC5ibs2uKwG6U-HSR7Hr5vPBbRovQ3lTHw-lV7yuV5TBikcuICu2BgBjMMy1hJq0-WHCcPohnupCWtUVEfxY2C71ryuOOcpn8xiL6cRCnp5cVGMEq868dKYdHyMHHYEo_eWFpODJcDGEG6T-PA7uzl5OpnKILVCwHYA_Kf10fo9B8Ox2pUZJGoYiZTVMeTv_HgDGRnrBCzyaihMsTe0w9BjyhogCukdf4cMa5-GZZnYYi8cBAcOb-fblNIi3P8sFswcersiu--x2DoIeTpoo0rUJ0p_KtcTKUJgaVQhmzztUnE1WLCBNpFqExHQ3WLvlOf4rXrXoJ_WSqR2QO0VV6T5yy6ab6In8j7YNOeZQPGPtqwP3l3C8JpUhMvWLcQjTGlssAtPHTNWFg.ZWXiHgDVt1yAKEP6tiF7rQ/formal_tokens.json"

In [20]:
# !wget "https://www.kaggleusercontent.com/kf/110519903/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..ntKOxmMLUsirKrm9CN7IIg.dj4QkBktFfEkqjQoyyI10W43vkc-iATvjs7ZCagqKctwjZZGzp_Az_-GEf7V6oTMe14S0_EIwYlhCKRm0EoPc9qwDtzDjRKxGYt9VKdlsuMlpjetk-FrnRCdIeR1BbwVttZL-q4wLiSF-C--37CdQoRyQQWUrwS7WxGkRScYzDeCCKlUcm5R5X4xXjeZ5vkhlTl0eGTPN6hXWK6gsvjtwEPWvQPz7VBk8_sipLJ0ztaY2InIQ7vFRfxeoJaBqzT331Utbgt1NMDYUlk8XbE_SBgT6ieUnlJ4UsXkeD1CdV9DU44oiuifGn6OPcP1iLEPosWjg0_D_10BearXXKlxS5d7a0u6os06Aslw48f7x8AQZDk4mqWnitHnrJPbF6ClLg99HhQtGtdTyXrvxsLB-gwJuf-a2Pj0dzkLi9Yg3n6rZ6SSxfl8qagj2M0uYAjsNWZu28rTu3lV11gt4TQ-L1qqxOEYq6vrXbsTbyFOCKQTCLRqVQexb5SLflWrlJRqZw-T1TbRitwckM5GJ1qvUZy8hbrsk1QaLHiPm-uyXve_pJH4BFMBB76VHASwFRRzLTQIzZ_LUEtMV1B2_w6tKpU-t3RVLmGR9ifdjCoI-oJPDwOKCEhxqStz7NM8d-PsOHG8kTZ6xo2URWRIAEBMjQ.BRICJUn0MlR8jAkZJvhDpQ/word2vec.model1668043951.0301144.model"
# !wget "https://www.kaggleusercontent.com/kf/110519903/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..VhSI42laxYSLf5k4WITZ8A.9mxYjzgj79TYJCx-C1k97M0lTq3GdnBAzKLqKBj52urwCfnWlSXY8G9wUwTxoH1Tzze7wsA7G5AweicR-qlwrUu6u7KTeEm8IwT-1nQyjjbGXlA7DjFvVJAwxNUqOodFtq7AVftu1pHzNPcxtUrR1_6dxHRn4tyN5-d7EylgG7AUeW16ee1F1PovMurS8nureMM4WlMnv65aSEfYwKWhZvzG77yt3bD8kbUzQzesopOdwkp0-5wAGXG50p5-B7GARO8AYEak5j_gUZpSBcKGX5dSWYPGWZaMQjaZvElSkpFqlymlGkF9a3KzOBKqiLFmFPpA-bPWmUEd2tGbfF-T-Jx5Tes4jOPOJErvsLBBUafwES4LWa_GUV_Codx_ysJrxoLGfR73UE7zE-WLazCV3NhAf5klQOn10CcWp3rBgYO1RYwLs7Vr1jKpqsOWRFUm_X02bBgekkVr8XGEDcNxhEulkP7u8I9NDIi_Ly_Z12_Kr6ymEhEQj4PjpQvC__ogpOQkjqNKcSrVOCFEVXYy_S7CBDCrU02gS6IiqM9f3QTMCb6Q3oE76yyXkKIxJ7NQ0atLvhMVu2tYsjoP6-0nyeSkaWRetvJlnzCE39BdL7PH06yX7APisFd3IFR0nwLB6QXo5YiFzV_pz73TRgSrDQ.VOGa4A0zsubveXIaelgxKQ/word2vec.model1668043951.0301144.model.syn1neg.npy"
# !wget "https://www.kaggleusercontent.com/kf/110519903/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..-X7Ep_oTxLWtiBIRR3pQzA.7QOw8U8FT31Kak6zb3h2yCSTVLMMiUBy0OXIiDdpEaygqFXRhBaMASj_bJG3tZtGeSQC1CSfnS5KRhOnKw6tLL7uH0LHBXKUFL2bjkX0YGWHZviFMZnJGuo-wj4aW6zYhVz2vvO4xOdja3dJhyVGTDBP862wCKM8EgtDEZLMwy5X3dHtRghJsmxt-CXPiirzmEPwYh6jZvQlF6KcZeKROmsUdIEeclrAoQhFgYrcrzA2q5mEGJAvo7dRu7PsDz_uMrgxUCF_ynkK1g7Lm8GkqmImM14V8RNlUSDwXESzBj9Ooo1k8YzAVvNa-sUPCqJJSOJuU7WH4Ts7CuUFxxIzi8QUF-OKljMdqwgA2Z-S19bjwp9Yrck3nscKU04B3wMlFWCU61KvmFdflxZNPqt_FCiPOy3lsAVikp6CJs638cMu7WkY6dz6LYZSJEWF_JtmBzzmTHIWmgx81_vJx-NgsBKolGbopDRNHnR_LErbDpxHdHoLF4tVMS2xrD1zU3b8oxrY1-TK6TXzaIpyJ_EiSOThnTwzhutmRsAnonJwMPP6-VB9g80qfMcbj8lStSFhAcJw51hcNxvRhJlwoqrxg16CmCSxbUqHHLrwFFvl1zozOocP1Kqr_Iw4DJJEJK7FOaWc9SbiFcDvd68kqcUYbA.ziO2A_aBGaimkoqSg0Q9XQ/word2vec.model1668043951.0301144.model.wv.vectors.npy"

In [21]:
# Word2VecLanguageModelService.model_path = "./word2vec.model1668043951.0301144.model"
# Word2VecLanguageModelService.formal_tokens_path = "./formal_tokens.json.1"
# wv_service = Word2VecLanguageModelService(load_model=True)

In [22]:
# import torch
# from transformers import BertTokenizer, BertForMaskedLM, pipeline, AdamW
# model_pretrained_name = "HooshvareLab/bert-fa-zwnj-base"
# _tokenizer = BertTokenizer.from_pretrained(model_pretrained_name)

In [23]:
# formal_tokens = set(wv_service.formal_tokens)
# print(len(formal_tokens), len(_tokenizer.vocab))
# formal_tokens.update([wv_service.provider.process_context(token)[0] if wv_service.provider.process_context(token)  else "#Digits#"  for token in _tokenizer.vocab])
# print(len(formal_tokens))
# with open("./formal_tokens_merged_with_bert.json", 'w') as file:
#     tokens_list = list(formal_tokens)
#     file.write(json.dumps(tokens_list, ensure_ascii=False))

In [24]:
# !wget "https://www.kaggleusercontent.com/kf/110519903/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..FZx5oFoqSQay7uEATb5d3Q.pXWPOoPXg163CYlK-Hd0ka_wX0JLv4FqOKsy-ascG-OKxzC0luTtKOJfYyXpNjNGJVlj0I9zi4E8pzHJj7MQSR5sQSdV0DJi8O1hvrYFUgUl4Wvv3kG_BrAoixvbo8D8d4UgBZl3A-06W1QOTWzqASCrFlwTHac3tNmO5mX-8Z1vGL-cGFeOFXARKL6p4ckJvA4Ylkc-OjF06AkSnslzpHgVzxHG-EAN5bOMT8YgljYztPgo50uB5RYpgOwopZR2AoS49a07BeCLnQ2Gaf6npjitOx7Y0ivwaVsP01L3nkVYD0BDmez_3m1s6q4ITQXM3oHTf_QgCbzybtKbGMbPAqyg8xH5jvKK0R83zxNi8nOjC8ztym1EqRPi6fmTzYKyp3_ROUX8q-6meGeS6arIF4ZvK5P-W3DHj9Cybz2pCbQkrxLs4hWqx8wKfK774eHl2zmCPevbBiz1iLOGQ32Y40bAElaDYwgla1lSMFrF8ZjgqMd8ptOCPpRelW4-Z8ggzElyFGgKeL9WeNFal0C9u4p0fI2D1GfSzXJNY32CQdfX9IWa_9OLL-YjkTw1Gy1riz_aR3tolSA8GUAeSeHWGiH_F17dnnNp3r3p_ukw0zy7wd6_k-mRo18_g1A3f3SU3CWGao23qn0ZPuxTnJLEGw.twCQiGgJchYrNxUjSkZKHA/word2vec.model1668043951.0301144.model"
# !wget "https://www.kaggleusercontent.com/kf/110519903/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..dORZS3Srf-2eymEqRNToDQ.LHwKbHmy2DsK0xw6HdIG6swk52C7hRYSMJVIJAbXfZWblbs7aOF2jCCemrahKbf3mTEHYQtpUdTzf9dcCH2kBgy5Blc7frRyHdzzaiAIMV1ueW_CevTL9b-Zc__ahvFf6w9gaw85-98kvcWNopcSj4mZFCv_t64fhN91oiZQWLyJXPaM9WtBwlJOEGEoYsqrDXoTddfPbZzuweLBdtWgnYUJBH5NF2pVLTIdkJjgxEKPkO1GXa6CEgIqfT2eOdhcOeP9RiH6ws7hPnDUV7Q7j4eCvLhC11lNBmKR7kTnbtDKow1JGMDwb0rDbdKjHfWqRVgXyiPCXOTQGFI1WlUXefkQgHQt05Zp9MVSRfa_4mUdSqoiMsASV3CXs2j8HZNIYOZFG_wRICMB4POfxY2DMpN4EtNr4UD-qZkrgOdR8WncuHCyobn16xLNEa0dGnQO5dMxk4qBe8lz5xE5gJYYRaR3kMns6UdfiaOE8xhlvflhoxoSmYY72CW4lS-6UHvePtn7CsAp9k4nYSCHOEqDMKJ5cPUoyfxeIprMVWX54JQCCcmZ8ccts5Kl26WndF5jLrGW7xVYuiNvTyO_LBtY6-Q3XZ81Tr4exJptzKFqSyy9TnunULl4xOUURqqGRKRsxZr6EV1x_wstYjsO_mOwSg.zsrLt7UB2rj2sUvaAyM1Eg/word2vec.model1668043951.0301144.model.syn1neg.npy"
# !wget "https://www.kaggleusercontent.com/kf/110519903/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..Uaq2ns_UXZyet23L_m9_Jw.CrUfsszWqGdSwpUs4LF5-JsveDQZ47ocf508cCz3M2CjnQDJoWLqDiywwl3kj7TAikOlbDs-ayenN83mNx0f-fWrDJs6p3MY39Nmj73ehBYqYzsqLXGssi6Cz_241DSXKzdtyv3l7DSNdd23HjSpt0SZYB9Sp2284XqjQG_QRPxGyEAPgpqBjz_u1q-orUz4PhNtK8k2kxcUyy-H0tyRGyQ32REHrv0lnK4ef5UessOdZEOJVHwwLRW6E2o-Txh3tt9opkso5rK1t7Ll2hDhBghkCN8vF_sQy4QSVZ6G-uPr8lMbIGhD_gQgSImpHGJsvnZHP-33PRInVCwLDF8yRsVdoa4sn1b2u5hTXL10i22iV1FkUPXCqUOqCATyFLc5pvLQzZ7N8EAsiyURZreEK7cEzQRltnSS9YZukm6ijNHbS5N9ZqCd-mRprPyHWEgr5EM87jUuRXiX_9ZDbShdWkJOc2W1O3tbC8PaosN-r4wEBYYSFCiHreJOsj2IM21iURQNjO5Xe1mpdbhFTp_RAU3fgjhlMfQiPJjhGDoadRqRK6E0fRg_grKWi-hEMP_x53y-dTEFaSitVxeiNhCNTSH-Q8N3lDzJkeGAZ0aTMgM4E8kwPrCjlm6foV4nThUwgXNRMi9BI6d_NGl5LA6Jig.EOik77NhftRPsiXeiUKTVQ/word2vec.model1668043951.0301144.model.wv.vectors.npy"

In [25]:
!wget "https://github.com/amirmosio/ColloquialWordProject/raw/main/language_model/kaggle_outputs/formal_tokens_merged_with_bert.json" 

--2022-11-14 22:19:25--  https://github.com/amirmosio/ColloquialWordProject/raw/main/language_model/kaggle_outputs/formal_tokens_merged_with_bert.json
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/amirmosio/ColloquialWordProject/main/language_model/kaggle_outputs/formal_tokens_merged_with_bert.json [following]
--2022-11-14 22:19:25--  https://raw.githubusercontent.com/amirmosio/ColloquialWordProject/main/language_model/kaggle_outputs/formal_tokens_merged_with_bert.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12012215 (11M) [text/plain]
Saving to: ‘formal_tokens_merged_with_bert.json’



In [26]:
Word2VecLanguageModelService.model_path_load = "../input/train-word2vec-for-one-epoch/word2vec.model1668043951.0301144.model"
Word2VecLanguageModelService.model_path_save = "./word2vec.model"
Word2VecLanguageModelService.formal_tokens_path_load = "./formal_tokens_merged_with_bert.json"
wv_service = Word2VecLanguageModelService(load_model=True)


In [27]:
def get_generator():
    all_tokens_generator = chain(
        wv_service.provider.bring_processed_colloquial_tokens(),
                                 wv_service.provider.bring_processed_formal_tokens(0),
                                 wv_service.provider.bring_processed_formal_tokens(1),
                                 wv_service.provider.bring_processed_formal_tokens(2),
                                 wv_service.provider.bring_processed_formal_tokens(3),
                                 wv_service.provider.bring_processed_formal_tokens(4)
    )
    return wv_service.shuffle(all_tokens_generator)

wv_service.model.min_count = 1
wv_service.save_model()
wv_service.model.build_vocab(SentencesIterator(get_generator), update=True)
wv_service.model.train(SentencesIterator(get_generator), epochs=1, total_examples=wv_service.model.corpus_count)
wv_service.save_model()

Saving model...
Saved.
10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000-10000